<a href="https://colab.research.google.com/github/whyzhuce/XConparraison/blob/master/Options_am%C3%A9ricaines_et_vol_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Options américaines : l'impact de la volatilité forward
---

> **Antonin Chaix**

Dans le cadre du modèle de Black & Scholes, on regarde dans ce colab comment la structure par terme de la volatilité de l'actif sous jacent, et donc en particulier sa volatilité forward, peut influencer le prix des options américaines.

Les options sont évaluées par différences finies, comme dans [ce colab plus général](https://colab.research.google.com/drive/190FfBc0w-fz8vim3x7RrFcb_Tu-Px2Hu?usp=sharing) (avec une volatilité locale sur l'actif, et où sont présentés tous les détails du schéma de différences finies utilisé ci-dessous).

On se donne deux dates $T_1<T_2$, et des volatilités implicites $\sigma_\text{imp}(T_1)$ et $\sigma_\text{imp}(T_2)$ pour ces deux maturités ; on souhaite étudier le prix des options européennes et américaines de maturité $T_2$ dans le modèle de Black & Scholes généralisé suivant :
$$
\frac{dS_t}{S_t} = (r - d)\, dt + \sigma(t)\,dW_t
$$
On suppose la fonction de volatilité instantanée $\sigma$ constante par morceaux : $\sigma(t) = \sigma_1$ sur $[0, T_1[$ et $\sigma(t) = \sigma_2$ sur $[T_1,, T_2[$, et on calibre $\sigma_1$ et $\sigma_2$ sur les deux volatilités implicites $\sigma_\text{imp}(T_1)$ et $\sigma_\text{imp}(T_2)$. Compte tenu de la relation $\sigma_\text{imp}(T)^2\;T = \int_0^{T}\sigma(t)^2dt$ entre la volatilité implicite et volatilité instantanée, on obtient :
$$
\sigma_1 = \sigma_\text{imp}(T_1) \qquad\text{et}\qquad \sigma_2 = \sqrt{\frac{\sigma_\text{imp}(T_2)^2\,T_2-\sigma_1^2\, T_1}{T_2-T_1}}
$$

Dans ce qui suit on va comparer les prix d'options obtenus dans le cas d'un BS standard ($\sigma_\text{imp}(T_1)=\sigma_\text{imp}(T_2)$ conduisant à une volatilité instantanée $\sigma(t)$ constante), et dans le cas où les deux volatilités implicites diffèrent, induisant dans le modèle une term structure non constante de volatilité.

## Imports et formules de BS

In [ ]:
import numpy as np
import math
import time
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
from scipy.stats import norm

# BS call function
def bs_call(T, K, F0, sigma) :
    sigma_sqrt_T = sigma * np.sqrt(T)
    d1 = (np.log(F0/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
    d2 = d1 - sigma_sqrt_T
    return F0 * norm.cdf(d1) - K * norm.cdf(d2)

# BS call function
def bs_put(T, K, F0, sigma) :
    return bs_call(T, K, F0, sigma) - (F0 - K)

## Implémentation des différences finies (Crank-Nicholson)

Cf. [ce colab](https://colab.research.google.com/drive/190FfBc0w-fz8vim3x7RrFcb_Tu-Px2Hu?usp=sharing) pour tous les détails sur le schéma de différences finies implémenté ci-dessous

In [ ]:
class bs_pde :

	def __init__(self, S0, r, d, sigma, T, nt, ns, nstdev):

		self.t = np.linspace(0., T, nt, endpoint=True)
		self.S0 = S0
		self.r = r
		self.d = d
		self.sigma = sigma

		# s discretization
		avg_vol = (sigma(0) + sigma(T)) / 2.
		u_max = math.exp(nstdev * avg_vol * math.sqrt(T))
		s_max = S0 * u_max
		s_min = S0 / u_max
		self.s = np.linspace(s_min, s_max, ns, endpoint=True)


	# compute & display PDE price
	def compute_price (self, payoff, name = 'PDE price', closed_form_price = -1, display = True, return_price_grid = False) :
		t_start = time.time()

		# Crank Nicholson
		theta = 0.5

      	# space & time dimensions
		nt = len(self.t)
		ns = len(self.s)
		ds = (self.s[-1] - self.s[0]) / (ns - 1.)
		dt = (self.t[-1] - self.t[0]) / (nt - 1.)

		# option value
		v = np.zeros((nt, ns))

		# payoff at final date
		v[-1,:] = payoff(self.t[-1], v[-1,:])

		# backward loop from t = t_max - dt to t = 0
		for i in range(nt-2, -1, -1): # => i = nt-2 ... 0

			# precalcs
			vol = self.sigma(self.t[i])
			var =  (vol*self.s)**2 * dt
			mu_dt = (self.r - self.d) * self.s * dt
			r_dt = self.r * dt

			alpha_u = (var + mu_dt * ds) / (2.*ds**2)
			alpha_d = (var - mu_dt * ds) / (2.*ds**2)
			alpha_c = - var / ds**2

			# limits
			alpha_u[0]  = mu_dt[0] / ds
			alpha_c[0]  = -alpha_u[0]
			alpha_d[0]  = 0.
			alpha_u[-1] = 0.
			alpha_c[-1] = mu_dt[-1] / ds
			alpha_d[-1] = -alpha_c[-1]

			# transition coefs
			p_ur = (1.-theta)*alpha_u
			p_dr = (1.-theta)*alpha_d
			p_cr = (1.-theta)*alpha_c + 1. - (1.-theta)*r_dt
			p_ul = -theta*alpha_u
			p_dl = -theta*alpha_d
			p_cl = -theta*alpha_c + 1. + theta*r_dt

			# compute right member
			right = np.empty(ns)
			right[0] = p_cr[0]*v[i+1, 0] + p_ur[0]*v[i+1, 1]
			right[1:ns-1] = p_dr[1:ns-1]*v[i+1,0:ns-2] + p_cr[1:ns-1]*v[i+1,1:ns-1] + p_ur[1:ns-1]*v[i+1,2:ns]
			right[-1] = p_cr[-1]*v[i+1,-1] + p_dr[-1]*v[i+1,-2]

			# backward step inverting tridiagonal matrix
			upper_band = np.insert(p_ul, 0, 0.)[:-1] # transform p_ul to call solve_banded (0 must be in first position)
			lower_band = np.append(p_dl[1:ns], 0)  # transform p_ul to call solve_banded (0 must be in last position)
			tridiag = np.array([upper_band, p_cl, lower_band])
			v[i,:] = solve_banded((1, 1), tridiag, right)

			# apply payoff to v
			v[i,:] = payoff(self.t[i], v[i,:])

		# interpolate price in option values array
		price_interp = interp1d(self.s, v[0,:], kind='linear')
		price = price_interp(self.S0)

		if (display):
			print("--------------------------------------------------------------")
			if (closed_form_price == -1) :
				print("{} = {:.5f}".format(name, price) )
			else :
				print("{} = {:.5f} (closed-form price = {:.5f})".format(name, price, closed_form_price) )
			if (closed_form_price != -1):
				print("Rel. error : {:.4f}%".format(100.*(price-closed_form_price)/closed_form_price))
			print("PDE computation time : {:.3f} seconds".format(time.time()-t_start))

		if (return_price_grid):
	 		return v
		else:
			return price



## Pricing des options


In [ ]:
# PDE params
T2 = 1 # scheme final date = options maturity date
T1 = T2/2
nt = 201
ns = 301
nstdev = 6

# Params du sous-jacent et des options
S0 = 100
K = 100
r = 0.05
q = 0.0
implied_vol = 0.20 # vol implicite de référence pour la maturité T2

# initialisation sigma1 et sigma2
sigma1 = sigma2 = 0

# Calibration de sigma1 et sigma2 sur les vols implicites
def calibrate_sigma():
    global sigma1, sigma2
    sigma1 = implied_vol_T1
    sigma2 = math.sqrt( (implied_vol_T2**2 * T2 - sigma1**2 * T1) / (T2 - T1) )

# Fonction de vol instantanée
def sigma(t):
    global sigma1, sigma2
    if (t < T1) :
        return sigma1
    else :
        return sigma2

# Définition des payoffs des options européennes et américaines
def european_call_payoff(t,v):
	if (math.isclose(t,T2)):
		v = np.maximum(pde.s - K, 0)
	return v

def european_put_payoff(t,v):
	if (math.isclose(t,T2)):
		v = np.maximum(K - pde.s, 0)
	return v

def american_call_payoff(t,v):
	return np.maximum(pde.s - K, v)

def american_put_payoff(t,v):
	return np.maximum(K - pde.s, v)

# Prix du call et du put européen de maturité T2
DF = math.exp(-r*T2)
fwd = S0 * math.exp((r-q)*T2)
call_eur =  DF * bs_call(T2, K, fwd, implied_vol)
put_eur  =  DF * bs_put(T2, K, fwd, implied_vol)

# Prix des options dans le modèle BS classique
# la vol implicite de maturité T1 est la même que la la vol implicite de maturité T2, la vol instantanée est constante au cours du temps
implied_vol_T2 = implied_vol_T1 = implied_vol
calibrate_sigma()
pde = bs_pde (S0, r, q, sigma, T2, nt, ns, nstdev)
print("\n")
print("Black & Scholes standard, volatilité constante = {:.2f}% ".format(implied_vol_T2*100) )
print("===================================================================================================================")
pde_call_eur_flat_vol = pde.compute_price(european_call_payoff, "Call européen - PDE", call_eur)
pde_call_us_flat_vol  = pde.compute_price(american_call_payoff, "Call américain - PDE")
pde_put_eur_flat_vol  = pde.compute_price(european_put_payoff, "Put européen - PDE", put_eur)
pde_put_us_flat_vol   = pde.compute_price(american_put_payoff, "Put américain - PDE")
print("\n")

# Prix des options dans le modèle BS généralisé avec une vol implicite croissante et donc une volatilité forward sigma2 plus élevée (sigma1 < implied_vol_T2 < sigma2)
implied_vol_T2 = implied_vol
implied_vol_T1 = implied_vol - 0.05
calibrate_sigma()
pde = bs_pde (S0, r, q, sigma, T2, nt, ns, nstdev)
print("Volatilité croissante : vol implicite T1 = {:.2f}%, vol implicite T2 = {:.2f}%, vol forward T1 -> T2 = {:.2f}%".format(implied_vol_T1*100, implied_vol_T2*100, sigma2*100) )
print("===================================================================================================================")
p = pde.compute_price(european_call_payoff, "Call européen - PDE", call_eur)
p = pde.compute_price(american_call_payoff, "Call américain - PDE")
p = pde.compute_price(european_put_payoff, "Put européen - PDE", put_eur)
p = pde.compute_price(american_put_payoff, "Put américain - PDE")
print("\n")

# Prix des options dans le modèle BS généralisé avec sigma1 > implied vol > sigma2 (vol forward faible)
implied_vol_T2 = implied_vol
implied_vol_T1 = implied_vol + 0.05
calibrate_sigma()
pde = bs_pde (S0, r, q, sigma, T2, nt, ns, nstdev)
print("Volatilité décroissante : vol implicite T1 = {:.2f}%, vol implicite T2 = {:.2f}%, vol forward T1 -> T2 = {:.2f}%".format(implied_vol_T1*100, implied_vol_T2*100, sigma2*100) )
print("===================================================================================================================")
p = pde.compute_price(european_call_payoff, "Call européen - PDE", call_eur)
p = pde.compute_price(american_call_payoff, "Call américain - PDE")
p = pde.compute_price(european_put_payoff, "Put européen - PDE", put_eur)
p = pde.compute_price(american_put_payoff, "Put américain - PDE")

print("\n")



Black & Scholes standard, volatilité constante = 20.00% 
--------------------------------------------------------------
Call européen - PDE = 10.45277 (closed-form price = 10.45058)
Rel. error : 0.0210%
PDE computation time : 0.022 seconds
--------------------------------------------------------------
Call américain - PDE = 10.45277
PDE computation time : 0.015 seconds
--------------------------------------------------------------
Put européen - PDE = 5.57572 (closed-form price = 5.57353)
Rel. error : 0.0393%
PDE computation time : 0.017 seconds
--------------------------------------------------------------
Put américain - PDE = 6.08911
PDE computation time : 0.017 seconds


Volatilité croissante : vol implicite T1 = 15.00%, vol implicite T2 = 20.00%, vol forward T1 -> T2 = 23.98%
--------------------------------------------------------------
Call européen - PDE = 10.44969 (closed-form price = 10.45058)
Rel. error : -0.0085%
PDE computation time : 0.008 seconds
----------------------

## Interprétation des résultats

Le modèle étant calibré sur la volatilité implicite de maturité $T_2$, le prix des call et put européens de maturité $T_2$ reste totalement inchangé quelle que soit la term structure de volatilité (autrement dit quelle que soit la vol implicite de maturité $T_1$).

De même, nous sommes ici dans le cas d'un taux d'intérêt positif et d'absence de dividendes sur l'actif, cas où le prix du call américain coïncide parfaitement avec celui du call européen, et ce quel que soit le modèle utilisé (explications détaillées dans [ce colab](https://colab.research.google.com/drive/1mV2OMYiAV64kpu1-zMJLye1X1h2RTec4)). En conséquence, pour les 3 term structures de volalité, le prix du call américain est égal à celui de l'européen et demeure inchangé.

En revanche, le prix du put américain est dans cette situation strictement supérieur à celui du put européen (car son exercice anticipé est parfois plus intéressant que sa conservation) et **le prix du put américain est sensible à la term structure de volatilité** :

* Prix du put américain pour une volatilité constante : **6.09**
* Prix du put américain pour une volatilité croissante : **5.87**
* Prix du put américain pour une volatilité décroissante : **6.47**

Autrement dit, **plus la volatilité forward est faible, plus le put américain est cher.**

Comment expliquer qualitativement ce phénomène ?

Le surplus de valeur du put américain par rapport au put européen s'explique par l'existence de trajectoires de l'actif où l'exercice anticipé du put européen est plus intéressant que la conservation de l'optionnalité pour une date ultérieure. Plus formellement, il s'agit des situations où à une date $t>0$, la valeur $K-S_t$ de l'exercice anticipé du put américain est supérieure à son prix $P_t$ à cette date.

Or la valeur $P_t$ en $t$ du put américain dépend de la volatilité forward de l'actif entre $t$ et la maturité $T_2$ de l'option : plus la volatilité en question est faible, plus le prix $P_t$ est faible, et donc plus exercice immédiat de valeur $K-S_t$ a de chance d'être préférable. Ce nombre accru d'exercices anticipés renchérit alors la valeur du put américain.
